In [1]:
from getEPH import *
import pandas as pd
#http://statsmodels.sourceforge.net/devel/examples/generated/example_wls.html
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)
np.random.seed(1024)
%matplotlib inline

NOTAS

* CALCULAR EN BASE AL MODELO DE CURVAS LAS DERIVADAS Y DONDE HACE EL PICO EN EDAD
* 


# DOWNLOAD DATA

In [ ]:
#get data
getEPHdbf('t310')

In [2]:
data1 = pd.read_csv('data/cleanDatat310.csv')

In [3]:
data2 = categorize(data1)
data3 = schoolYears(data2)
data = make_dummy(data3)

In [4]:
data.head()

,CODUSU,NRO_HOGAR,COMPONENTE,AGLOMERADO,PONDERA,familyRelation,female,age,schoolLevel,finishedYear,...,P47T,P21,primary,secondary,university,male_14to24,male_25to34,female_14to24,female_25to34,female_35more
0,302468,1,1,32,1287,1,1,20,7.0,2,...,2000,0,7.0,5.0,1.0,0,0,1,0,0
1,302468,1,2,32,1287,10,1,20,6.0,2,...,2000,0,7.0,5.0,1.0,0,0,1,0,0
2,307861,1,1,32,1674,1,0,42,2.0,1,...,3000,3000,7.0,0.0,0.0,0,0,0,0,0
3,307861,1,2,32,1674,2,1,44,7.0,1,...,2800,2800,7.0,5.0,5.0,0,0,0,0,1
4,307861,1,3,32,1674,3,0,13,4.0,2,...,0,0,7.0,0.0,0.0,0,0,0,0,0


# NEW VARIABLES FOR MODEL

In [5]:
#select variables 
dataModel = data.copy().loc[:,
                              ['PONDERA','P47T','P21',
                               'primary','secondary','university',
                               'male_14to24','male_25to34',
                               'female_14to24', 'female_25to34', 'female_35more',
                               'female','age']]

In [6]:
#Make polynomial variables and education as a total
dataModel['education'] = dataModel.primary + dataModel.secondary + dataModel.university
dataModel['education2'] = dataModel['education'] ** 2
dataModel['age2'] = dataModel['age'] ** 2

dataModel.P47T.replace(to_replace=[0], value=[1] , inplace=True, axis=None)
dataModel.P21.replace(to_replace=[0], value=[1] , inplace=True, axis=None)
dataModel['lnIncome']= np.log(data.P21)
dataModel['lnIncomeT']= np.log(data.P47T)
dataModel.dropna(inplace=True)
dataModel.head()


,PONDERA,P47T,P21,primary,secondary,university,male_14to24,male_25to34,female_14to24,female_25to34,female_35more,female,age,education,education2,age2,lnIncome,lnIncomeT
0,1287,2000,1,7.0,5.0,1.0,0,0,1,0,0,1,20,13.0,169.0,400,-inf,7.600902
1,1287,2000,1,7.0,5.0,1.0,0,0,1,0,0,1,20,13.0,169.0,400,-inf,7.600902
2,1674,3000,3000,7.0,0.0,0.0,0,0,0,0,0,0,42,7.0,49.0,1764,8.006368,8.006368
3,1674,2800,2800,7.0,5.0,5.0,0,0,0,0,1,1,44,17.0,289.0,1936,7.937375,7.937375
4,1674,1,1,7.0,0.0,0.0,0,0,0,0,0,0,13,7.0,49.0,169,-inf,-inf


In [7]:
#Select only people with jobs and income for that job
jobsAndIncome = ((data.activity==1) & (data.P21>1))
dataModel = dataModel.copy().loc[jobsAndIncome,:]

# Graficos exploratorios

In [ ]:
fig = plt.figure(figsize=(16,12))
ax1 = fig.add_subplot(2,2,1)
ax2 = fig.add_subplot(2,2,2)
ax3 = fig.add_subplot(2,2,3)
ax4 = fig.add_subplot(2,2,4)

ax1.plot(dataModel.education,dataModel.P47T,'ro')
ax1.set_ylabel('Ingreso total')
ax1.set_xlabel('Educacion')
ax2.plot(dataModel.age,dataModel.P47T,'ro')
ax2.set_xlabel('Edad')
ax3.plot(dataModel.education,dataModel.P21,'bo')
ax3.set_ylabel('Ingreso Laboral')
ax3.set_xlabel('Educacion')
ax4.plot(dataModel.age,dataModel.P21,'bo')
ax4.set_xlabel('Edad')

In [ ]:
fig = plt.figure(figsize=(16,12))
ax1 = fig.add_subplot(2,2,1)
ax2 = fig.add_subplot(2,2,2)
ax3 = fig.add_subplot(2,2,3)
ax4 = fig.add_subplot(2,2,4)


sns.kdeplot(dataModel.P47T,ax=ax1,color = 'red')
sns.kdeplot(dataModel.lnIncomeT,ax=ax2,color = 'red')
sns.kdeplot(dataModel.P21,ax=ax3)
sns.kdeplot(dataModel.lnIncome,ax=ax4)

# PLOTS FOR LnINCOME ~ EDUC AND AGE

In [ ]:
g = sns.JointGrid(x="education", y="lnIncome", data=dataModel)  
g.plot_joint(sns.regplot, order=2)  
g.plot_marginals(sns.distplot)

g2 = sns.JointGrid(x="age", y="lnIncome", data=dataModel)  
g2.plot_joint(sns.regplot, order=2)  
g2.plot_marginals(sns.distplot)

# Modelos

Tomo el de mejor performance para evaluar en el test set. Basicamente son dos posibiliades INDEC o ALTERNATIVO (que habiamos propuesto no cortar las edades y los años de escolaridad, sino usar las variables y directamente usar el cuadrado). Cada uno lo pruebo con ingresos laborales (con y sin constante) y con el log del ingreso laboral.

## 1 CEPAL con ingresos laborales

In [8]:
dataModel1 = runModel(dataModel, income = 'P21')

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.704
Model:                            WLS   Adj. R-squared:                  0.703
Method:                 Least Squares   F-statistic:                     939.2
Date:                Wed, 16 Nov 2016   Prob (F-statistic):               0.00
Time:                        19:26:27   Log-Likelihood:                -31770.
No. Observations:                3568   AIC:                         6.356e+04
Df Residuals:                    3559   BIC:                         6.361e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1651.4434    222.230      7.431      0.0

## 2 - CEPAL con Log ingresos laborales 

In [9]:
dataModel2 = runModel(dataModel, income = 'lnIncome')

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.990
Model:                            WLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 4.029e+04
Date:                Wed, 16 Nov 2016   Prob (F-statistic):               0.00
Time:                        19:26:33   Log-Likelihood:                -4067.6
No. Observations:                3568   AIC:                             8153.
Df Residuals:                    3559   BIC:                             8209.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          6.8445      0.094     72.519      0.0

## 3 - CEPAL con ingresos totales

In [10]:
dataModel3 = runModel(dataModel, income = 'P47T')

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.693
Model:                            WLS   Adj. R-squared:                  0.692
Method:                 Least Squares   F-statistic:                     893.6
Date:                Wed, 16 Nov 2016   Prob (F-statistic):               0.00
Time:                        19:28:30   Log-Likelihood:                -32497.
No. Observations:                3568   AIC:                         6.501e+04
Df Residuals:                    3559   BIC:                         6.507e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1939.5764    272.446      7.119      0.0

## 4 - CEPAL con Log ingresos totales

In [12]:
dataModel4 = runModel(dataModel, income = 'lnIncomeT')

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.992
Model:                            WLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                 4.615e+04
Date:                Wed, 16 Nov 2016   Prob (F-statistic):               0.00
Time:                        19:29:21   Log-Likelihood:                -3913.3
No. Observations:                3568   AIC:                             7845.
Df Residuals:                    3559   BIC:                             7900.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          7.1741      0.090     79.371      0.0

## 5 - ALTERNATIVO con Log ingresos totales

In [13]:
dataModel5 = runModel(dataModel, income = 'lnIncomeT', variables=['education','education2',
                                'age','age2','female'])

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.992
Model:                            WLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                 6.988e+04
Date:                Wed, 16 Nov 2016   Prob (F-statistic):               0.00
Time:                        19:30:48   Log-Likelihood:                -3897.9
No. Observations:                3568   AIC:                             7808.
Df Residuals:                    3562   BIC:                             7845.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          6.0742      0.133     45.552      0.0

## 6 - ALTERNATIVO con log Income laboral 

In [14]:
dataModel6 = runModel(dataModel, income = 'lnIncome', variables=['education','education2',
                                'age','age2','female'])

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.990
Model:                            WLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 6.164e+04
Date:                Wed, 16 Nov 2016   Prob (F-statistic):               0.00
Time:                        19:32:27   Log-Likelihood:                -4034.5
No. Observations:                3568   AIC:                             8081.
Df Residuals:                    3562   BIC:                             8118.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.5698      0.139     40.200      0.0